# Import dataset

In [1]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

train_set = tfds.load('imdb_reviews', split='train', as_supervised=True).take(5000)
test_set = tfds.load('imdb_reviews', split='test', as_supervised=True).take(1000)

# Prepare tokens

In [2]:
max_text_length = 0

# training
X_train = []
y_train = []
for i, j in train_set:
    i = str(i.numpy())
    max_text_length = max(max_text_length, len(i))
    X_train.append(i)
    y_train.append(int(j))

tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_text_length, padding='post')

X_train = np.array(X_train)
y_train = np.array(y_train)

# testing
X_test = []
y_test = []
for i, j in test_set:
    i = str(i.numpy())
    X_test.append(i)
    y_test.append(int(j))

X_test = tokenizer.texts_to_sequences(X_test)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_text_length, padding='post')

X_test = np.array(X_test)
y_test = np.array(y_test)

# Build the RNN
We need to use an RNN to help predict positive/negative reviews because the order of words obviously matters in sentences. An embedding layer is needed to convert sequences of tokens into sequences of vectors that can be easily understood by the RNN layer.

In [3]:
model = keras.Sequential()
model.add(keras.layers.Embedding(10001, 64, mask_zero=True))
model.add(keras.layers.SimpleRNN(128, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.BinaryAccuracy()])

# Train the model

In [4]:
epochs = 5

model.fit(X_train, y_train, epochs=epochs, validation_split=0.1, batch_size=256)

Epoch 1/5
18/18 [==============================] - 816s 45s/step - loss: 0.6926 - binary_accuracy: 0.5149 - val_loss: 0.6922 - val_binary_accuracy: 0.5140
Epoch 2/5
18/18 [==============================] - 805s 45s/step - loss: 0.6766 - binary_accuracy: 0.6829 - val_loss: 0.6765 - val_binary_accuracy: 0.5800
Epoch 3/5
18/18 [==============================] - 817s 45s/step - loss: 0.6168 - binary_accuracy: 0.7264 - val_loss: 0.6443 - val_binary_accuracy: 0.6280
Epoch 4/5
18/18 [==============================] - 821s 46s/step - loss: 0.5635 - binary_accuracy: 0.7320 - val_loss: 0.6643 - val_binary_accuracy: 0.5760
Epoch 5/5
18/18 [==============================] - 822s 46s/step - loss: 0.4797 - binary_accuracy: 0.8267 - val_loss: 0.6340 - val_binary_accuracy: 0.6620


# Evaluate on test set

In [5]:
print(model.metrics_names)
print(model.evaluate(X_test, y_test))

['loss', 'binary_accuracy']
32/32 [==============================] - 53s 2s/step - loss: 0.6246 - binary_accuracy: 0.6770
[0.6245537996292114, 0.6769999861717224]


# Create new model with LSTM

In [4]:
model2 = keras.Sequential()
model2.add(keras.layers.Embedding(10001, 64, mask_zero=True))
model2.add(keras.layers.LSTM(128, activation='relu'))
model2.add(keras.layers.Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.BinaryAccuracy()])

# Train the new model

In [ ]:
epochs = 5

model2.fit(X_train, y_train, epochs=epochs, validation_split=0.1, batch_size=256)

Epoch 1/5


# Evaluate LSTM on test set

In [ ]:
print(model2.metrics_names)
print(model2.evaluate(X_test, y_test))